# Using a Random Forest Regressor to Find Most Influential Features

In [1]:
import pandas as pd

domain_dataset = pd.read_parquet("../data/curated/features_domain.parquet")

In [2]:
domain_dataset.head(10)

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,...,P20/P50,P10/P50,Gini coefficient,Top 1%,Top 5%,Top 10%,Lowest Quartile,Second Quartile,Third Quartile,Highest Quartile
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.774273,145.181126,4.0,3.0,2.0,9125.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
1,https://www.domain.com.au/7-pine-ridge-donvale...,$750 per week,"7 Pine Ridge, Donvale VIC 3111",House,-37.791251,145.175649,4.0,2.0,0.0,3259.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
2,https://www.domain.com.au/20-mulsanne-way-donv...,$1300 per week,"20 Mulsanne Way, Donvale VIC 3111",House,-37.797232,145.181264,5.0,2.0,2.0,5649.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
3,https://www.domain.com.au/3-monterey-crescent-...,$825pw / $3585pcm,"3 Monterey Crescent, Donvale VIC 3111",House,-37.792402,145.174323,3.0,1.0,5.0,3585.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
4,https://www.domain.com.au/3-49-leslie-street-d...,$680.00,"3/49 Leslie Street, Donvale VIC 3111",Townhouse,-37.781012,145.180705,3.0,2.0,2.0,2955.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
5,https://www.domain.com.au/6-81-97-mitcham-road...,$575.00,"6/81-97 Mitcham Road, Donvale VIC 3111",Townhouse,-37.797814,145.181397,2.0,1.0,2.0,2499.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
6,https://www.domain.com.au/4-49-leslie-street-d...,$680.00,"4/49 Leslie Street, Donvale VIC 3111",Townhouse,-37.780842,145.180724,3.0,2.0,2.0,2955.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
7,https://www.domain.com.au/10-51-55-leslie-stre...,$500 Per Week,"10/51-55 Leslie Street, Donvale VIC 3111",Apartment / Unit / Flat,-37.781431,145.181474,2.0,1.0,1.0,2173.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
8,https://www.domain.com.au/6-martha-street-donv...,$695 per week,"6 Martha Street, Donvale VIC 3111",Apartment / Unit / Flat,-37.795872,145.174319,3.0,2.0,1.0,3020.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4
9,https://www.domain.com.au/55-darvall-street-do...,$800.00,"55 Darvall Street, Donvale VIC 3111",House,-37.802745,145.175619,4.0,2.0,2.0,3476.0,...,0.32,0.11,0.555,13.1,27.9,39.7,28.5,22.1,19.0,30.4


In [3]:

import pandas as pd

# List of columns you want to drop
columns_to_drop = ['bond', 'Facility Count', 'price', 'index_right', 'chg_flag', 'chg_lbl', 'property_id', 'nearest_hospital_id', 'nearest_station_id', 'latitude', 'longitude']  # Replace with actual column names

# Dropping the columns
domain_dataset = domain_dataset.drop(columns_to_drop, axis=1)


In [4]:
import regex
domain_dataset.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in domain_dataset.columns.values]

In [5]:
import pandas as pd

# Calculating the number of missing values for each feature
missing_values = domain_dataset.isnull().sum()

# Filtering out features that have missing values
missing_values = missing_values[missing_values > 0]

# Displaying the features and their counts of missing values
print(missing_values)


Beds                            136
Baths                            67
Parking                           9
train_station_distance_km         2
mean_stores                    5373
total_stores                   5373
count_shopping_centres         5373
ERP_2021                          2
ERP_2026                          2
ERP_2031                          2
population_growth_2021_2026       2
population_growth_2026_2031       2
population_growth_2031_2036       2
total_population_growth           2
total_value_of_items           3853
total_number_of_items          3853
crime_frequency                3853
P80/P20                           5
P80/P50                           5
P20/P50                           5
P10/P50                           5
Gini coefficient                  5
Top 1%                            5
Top 5%                            5
Top 10%                           5
Lowest Quartile                   2
Second Quartile                   2
Third Quartile              

In [6]:
import pandas as pd

# Assuming 'domain_dataset' is your DataFrame and already loaded

# Columns you want to impute with 0
columns_to_impute = ['mean_stores', 'total_stores', 'count_shopping_centres', 'total_value_of_items', 'total_number_of_items', 'crime_frequency']

# Imputing missing values with 0 for these specific columns
domain_dataset[columns_to_impute] = domain_dataset[columns_to_impute].fillna(0)

# Check to confirm that the imputation was successful
print(domain_dataset[columns_to_impute].isnull().sum())


mean_stores               0
total_stores              0
count_shopping_centres    0
total_value_of_items      0
total_number_of_items     0
crime_frequency           0
dtype: int64


In [7]:
import pandas as pd

# Assuming 'domain_dataset' is your DataFrame and already loaded

# Calculating the number of missing values for each feature
missing_values = domain_dataset.isnull().sum()

# Filtering out features that have missing values
missing_values = missing_values[missing_values > 0]

# Displaying the features and their counts of missing values
print(missing_values)


Beds                           136
Baths                           67
Parking                          9
train_station_distance_km        2
ERP_2021                         2
ERP_2026                         2
ERP_2031                         2
population_growth_2021_2026      2
population_growth_2026_2031      2
population_growth_2031_2036      2
total_population_growth          2
P80/P20                          5
P80/P50                          5
P20/P50                          5
P10/P50                          5
Gini coefficient                 5
Top 1%                           5
Top 5%                           5
Top 10%                          5
Lowest Quartile                  2
Second Quartile                  2
Third Quartile                   2
Highest Quartile                 2
dtype: int64


In [8]:
print(domain_dataset.columns)

Index(['url', 'address', 'property_type', 'Beds', 'Baths', 'Parking',
       'extracted_price', 'geometry', 'sa2_code', 'sa2_name',
       ...
       'P20/P50', 'P10/P50', 'Gini coefficient', 'Top 1%', 'Top 5%', 'Top 10%',
       'Lowest Quartile', 'Second Quartile', 'Third Quartile',
       'Highest Quartile'],
      dtype='object', length=103)


In [9]:
# Dropping rows with any NaN values
cleaned_df = domain_dataset.dropna()
print(len(cleaned_df))

9422


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming 'domain_dataset' is your DataFrame
corr_matrix = cleaned_df.corr()

# Extract the correlations with the target variable
target_corr = corr_matrix['extracted_price'].abs().sort_values(ascending=False)

# Select the top 50 features for initial analysis, including your target
top_features = target_corr.head(70).index

print(top_features)

Index(['extracted_price', 'Baths', 'Beds', 'Second Quartile',
       'Gini coefficient', 'Top 10%', 'Top 5%', 'Mean', 'Highest Quartile',
       'Third Quartile', 'Top 1%', 'P80/P50', 'Sum', 'SA2 code_x', 'sa3_code',
       'sa4_code', 'Parking', 'P80/P20', 'P20/P50', 'cbd_distance_km',
       'pop_change_natural_increase', 'distance_to_closest_independent_school',
       'nearest_hospital_distance', 'Median', 'Median age of earners',
       'HHS_2036', 'HHS_2031', 'HHS_2026', 'P10/P50', 'HHS_2021',
       'population_growth_2026_2031', 'areasqkm', 'area_km2',
       'population_growth_2031_2036', 'total_population_growth', 'OCC_2036',
       'OCC_2031', 'OCC_2026', 'ERP_2036', 'POPD_2036',
       'nearest_parkres_distance', 'ERP_2031', 'population_growth_2021_2026',
       'Earners', 'OCC_2021', 'POPD_2031', 'pop_change_net_internal_migration',
       'closest_independent_school', 'population_density_persons_per_km2_2023',
       'PNPD_2036', 'SPD_2021', 'PNPD_2031', 'PNPD_2026', 'OPD

/var/folders/9c/2vbsxdnd7k7_tnzhl33znmpm0000gp/T/ipykernel_38301/2287984598.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = cleaned_df.corr()


In [11]:
# Create a smaller correlation matrix for these top features
top_corr_matrix = corr_matrix.loc[top_features, top_features]

# Target number of features to retain
target_feature_count = 50

# List to keep track of features to drop
to_drop = []

while len(top_features) - len(to_drop) > target_feature_count:
    # Compute the current reduced correlation matrix
    current_corr_matrix = top_corr_matrix.drop(index=to_drop, columns=to_drop)
    # Calculate the mean absolute correlation of each feature with others
    mean_abs_corr = current_corr_matrix.apply(lambda x: x.abs().drop(x.name).mean(), axis=0)
    # Find the feature with the highest mean correlation
    max_corr_feature = mean_abs_corr.idxmax()
    to_drop.append(max_corr_feature)

# Remaining features after dropping
remaining_features = [feature for feature in top_features if feature not in to_drop]

# Final DataFrame with reduced feature set
final_corr_matrix = corr_matrix.loc[remaining_features, remaining_features]

'''# Visualizing the final set of features with a heatmap
plt.figure(figsize=(18, 16))
sns.heatmap(final_corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Heatmap of Top 30 Features with Controlled Inter-Correlation')
plt.show()'''

'# Visualizing the final set of features with a heatmap\nplt.figure(figsize=(18, 16))\nsns.heatmap(final_corr_matrix, annot=True, fmt=".2f", cmap=\'coolwarm\')\nplt.title(\'Heatmap of Top 30 Features with Controlled Inter-Correlation\')\nplt.show()'

In [12]:
beds_baths_corr = domain_dataset[['Beds', 'Baths']].corr()

# Print the correlation value
print("Correlation between Beds and Baths:")
print(beds_baths_corr)

Correlation between Beds and Baths:
          Beds    Baths
Beds   1.00000  0.62069
Baths  0.62069  1.00000


In [13]:
# Assuming 'domain_dataset' is your DataFrame and 'remaining_features' contains the list of feature names to retain
reduced_dataset = cleaned_df[remaining_features]

# Optionally, include the target variable if it's not already in 'remaining_features'
if 'extracted_price' not in remaining_features:
    reduced_dataset['extracted_price'] = cleaned_df['extracted_price']

# Now 'reduced_dataset' contains only the features of interest
print(reduced_dataset.head())  # Displaying the first few rows of the new DataFrame to verify


   extracted_price  Baths  Beds  Second Quartile  Top 5%   Mean  \
0           1400.0    3.0   4.0             22.1    27.9  83728   
1            750.0    2.0   4.0             22.1    27.9  83728   
2           1300.0    2.0   5.0             22.1    27.9  83728   
3            825.0    1.0   3.0             22.1    27.9  83728   
4            680.0    2.0   3.0             22.1    27.9  83728   

   Highest Quartile  Top 1%        Sum  SA2 code_x  ...     POPD_2026  \
0              30.4    13.1  872950324   211021261  ...  16462.532778   
1              30.4    13.1  872950324   211021261  ...  16462.532778   
2              30.4    13.1  872950324   211021261  ...  16462.532778   
3              30.4    13.1  872950324   211021261  ...  16462.532778   
4              30.4    13.1  872950324   211021261  ...  16462.532778   

   closest_school  erp_2023_no.  Lowest Quartile  OPD_2036  crime_frequency  \
0            1759         16511             28.5   6251.52              0.0   


In [14]:
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming 'domain_dataset' is your DataFrame and already loaded
numeric_df = reduced_dataset.select_dtypes(include=[np.number])
# Separating the independent variables and the dependent variable
X = numeric_df.drop('extracted_price', axis=1)
y = numeric_df['extracted_price']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fitting the model
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Calculating the mean squared error
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

# Getting feature importances
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X_train.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)

# Displaying the top 10 most important features
print(feature_importances.head(10))


Root Mean Squared Error: 123.98690051235113
                                         importance
Baths                                      0.206236
Beds                                       0.156045
SA2 code_x                                 0.075155
Top 5%                                     0.070202
sa3_code                                   0.056827
nearest_hospital_distance                  0.043782
distance_to_closest_school                 0.042806
distance_to_closest_independent_school     0.038848
nearest_parkres_distance                   0.037958
population_density_persons_per_km2_2023    0.022666


In [15]:
feature_importances.to_csv("../data/curated/RF_feature_importance.csv")